In [9]:
%run classesDefinations.ipynb

In [10]:
CAPACITY = 10
BUS_ORIGIN = Cords(35.8942679, 14.5086503) #CORDS OF THE 15 VALLETTA BUS BAY)
BUS_STOP_DATA_PATH = "./Data/BusStopsMalta/export.json"

In [11]:
from datetime import datetime, timedelta
import json
import random

def divide_time_interval(n):
    """
    Divides the time interval between 8:00 AM and Midnight into 'n' equal intervals.

    Parameters:
    n (int): Number of intervals to divide the time into.

    Returns:
    list: A list of time intervals.
    """
    
    # Define start and end times
    start_time = datetime.strptime("08:00 AM", "%I:%M %p")
    end_time = datetime.strptime("12:00 AM", "%I:%M %p")

    # Calculate total duration in seconds
    total_duration = (end_time - start_time).seconds

    # Calculate duration of each interval in seconds
    interval_duration = total_duration / n

    # Generate time intervals
    intervals = []
    for i in range(n):
        interval_start = start_time + timedelta(seconds=i * interval_duration)
        intervals.append(interval_start.time())

    return intervals

In [16]:
class BusHandler():
    def __init__(self, vehicleAmount, numberOfRequests):
        self.vehicles = self.initVehiles(vehicleAmount)
        self.requests = self.initRequests(numberOfRequests)
        self.currentRequest = self.requests[0]
        self.currentRequestIndex = 0
        self.rejectedRequests = []

    def __str__ (self):
        stringOutput = "==========================================\n Vehicles: \n"
        for vehile in self.vehicles:
            stringOutput += str(vehile) + "\n"
        stringOutput += "==========================================\n Requests: \n"
        for request in self.requests:
            stringOutput += str(request) + "\n"
        stringOutput += "==========================================\n"
        return stringOutput

    def initVehiles(self, vehicleAmount):
        vehicles = []
        for i in range(vehicleAmount):
            vehicles.append(Vehicle(i, CAPACITY, BUS_ORIGIN))
        return vehicles

    def initRequests(self, numberOfRequests):
        requestId = 0
        timeIntervals = divide_time_interval(numberOfRequests)
        f = open(BUS_STOP_DATA_PATH, "r")
        data = json.load(f)
        elements = data["elements"]
        elementsLen = len(elements)
        requestList = []
        for i in range(numberOfRequests):
            #Get two random indexes from all the bus stops 
            r1 = random.randint(0, elementsLen-1)
            r2 = random.randint(0, elementsLen-1)
            while r1 == r2:
                r2 = random.randint(0, elementsLen-1)
            
            #Get the coordinates of the two bus stops
            lat1 = elements[r1]["lat"]
            lon1 = elements[r1]["lon"]
            lat2 = elements[r2]["lat"]
            lon2 = elements[r2]["lon"]

            #Create the request Cords
            reqPickup = Cords(lat1, lon1)
            reqDropoff = Cords(lat2, lon2)

            #Create the request
            requestList.append(Request(requestId, reqPickup, reqDropoff, timeIntervals[i],random.randint(1, 3)))

            #Increment the requestId
            requestId += 1
        f.close()
        return requestList
    
    def rejectRequest(self):
        #Error Checking
        if(self.currentRequestIndex >= len(self.requests)):
            raise ValueError("No more requests to reject, index exceeded the requests list length.")

        #Reject the current request and move to the next one
        self.rejectedRequests.append(self.currentRequest)
        self.currentRequestIndex += 1
        if self.currentRequestIndex < len(self.requests):
            self.currentRequest = self.requests[self.currentRequestIndex]
            return False
        else:
            return True
        
    def getCurrentRequest(self):
        return self.currentRequest

In [22]:
busHandler = BusHandler(5, 10)
print(busHandler)

 Vehicles: 
Vehicle: id: 0, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 1, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 2, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 3, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 4, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
 Requests: 
Request: id: 0, passengerAmount: 2, time: 08:00:00, origin: Req_Id: 0, lat: 35.9067149, long: 14.4380086, True, destination: Req_Id: 0, lat: 35.8556994, long: 14.4869488, False,
Request: id: 1, passengerAmount: 3, time: 09:36:00, origin: Req_Id: 1, lat: 35.887642, long: 14.4931662, True, destination: Req_Id: 1, lat: 35.9406225, long: 14.3559095, False,
Request: id: 2, passengerAmount: 1, time: 11:12:00, origin: Req_Id: 2, lat: 35.8496894, long: 14